In [1]:
import numpy
from rbatools.rba_problem_matrix import ProblemMatrix
from rbatools.rba_lp import LinearProblem
from scipy.sparse import coo_matrix

# Solving and manipulating a simple linear optimization problem with rbatools

#### For instructional purposes we use the example-LP provided here:
https://developers.google.com/optimization/lp/lp_example

### Maximize 3x + 4y 
#### s.t.
### x + 2y ≤ 14
### 3x - y ≥ 0
### x - y ≤ 2

    

## Construct solvable LP structure

In [2]:
print(ProblemMatrix.__doc__)


    Class holding RBA-Linear Problem.
    Required Object type for linear problems (matrices), to be used in this RBA-API.
    Should include all fields, required for a linear problem in the COBRA format, as attributes.

    Attributes
    ----------
    A : scipy.sparse.coo_matrix
        Lefthandside of constraint Matrix (aka Constraint Matrix)
    b : numpy.array
        Righthandside of Constraint Matrix
    row_signs : list
        Type of constraints ('E' for equality, 'L' for lower-or-equal inequality), 'G' for larger-or-equal inequality) --> Ax=b or Ax<=b or Ax>=b
    f : numyp.array
        Objective function linear get_coefficients
    LB : numpy.array
        Lower bounds of decision-variables
    UB : numpy.array
        Upper bounds of decision-variables
    row_names : list
        Names of constraints
    col_names : list
        Names of decision-variables
    rowIndicesMap : dict
        Dictionary mapping constraint names to their numeric index (generated automatical

### Initiate empty Matrix object

In [3]:
Example_LP_matrix=ProblemMatrix()

### Define lefthand side of LP (constraints) 

In [4]:
coeffs_constraint_1=[1,2]
coeffs_constraint_2=[3,-1]
coeffs_constraint_3=[1,-1]
Example_LP_matrix.A=coo_matrix(numpy.array([coeffs_constraint_1,coeffs_constraint_2,coeffs_constraint_3]))

### Define righthand side of LP

In [5]:
Example_LP_matrix.b=numpy.array([14,0,2])

### Define constraint types

In [6]:
Example_LP_matrix.row_signs=['L','G','L']

### Define objective function 

In [7]:
Example_LP_matrix.f=numpy.array([3,4])

### Define variable bounds

In [8]:
Example_LP_matrix.LB=numpy.array([0,0])
Example_LP_matrix.UB=numpy.array([10000,10000])

### Define variable and constraint IDs (columns and rows)

In [9]:
Example_LP_matrix.col_names=['x','y']
Example_LP_matrix.row_names=['c1','c2','c3']

### Initiate empty LP object and import matrix

In [10]:
print(LinearProblem.__doc__)



    Attributes
    ----------
    A : scipy.sparse.coo_matrix
        Lefthandside of constraint Matrix (aka Constraint Matrix)
    b : numpy.array
        Righthandside of Constraint Matrix
    row_signs : list
        Type of constraints ('E' for equality, 'L' for lower-or-equal inequality), 'G' for larger-or-equal inequality) --> Ax=b or Ax<=b or Ax>=b
    f : numyp.array
        Objective function linear get_coefficients
    LB : numpy.array
        Lower bounds of decision-variables
    UB : numpy.array
        Upper bounds of decision-variables
    row_names : list
        Names of constraints
    col_names : list
        Names of decision-variables
    rowIndicesMap : dict
        Dictionary mapping constraint names to their numeric index (generated automatically)
    colIndicesMap : dict
        Dictionary mapping variable names to their numeric index (generated automatically)
    lp_solver : str
        Selected linear problem solver.
    _lp_solver : rbatools.rba_lp._Solver

In [11]:
Example_LP=LinearProblem(lp_solver='swiglpk') # swiglpk or cplex
Example_LP.load_matrix(matrix=Example_LP_matrix)

### Build solvable LP-structure

In [12]:
lp_built=Example_LP.build_lp() 
if lp_built:
    print("LP built")

LP built


## Working with LP

### Optimize LP

In [13]:
Example_LP.solve_lp()

### Get solution status

In [14]:
Example_LP.return_solution_status()

'optimal'

### Get optimal objective value

In [15]:
Example_LP.return_objective_value()

0.0

### Get solution vector

In [16]:
Example_LP.return_primal_values()

{'x': 0.0, 'y': 0.0}

### Get shadow prices (dual values)

In [17]:
Example_LP.return_dual_values()

{'c1': 0.0, 'c2': 0.0, 'c3': 0.0}

### There must be a problem, since objective value is 0

The LP is defined as an maximization of the objective function. But by default rbatools assumes minimization.
Therefore the obtained solution corresponds to the minimal feasible objective value.
#### In order to fix the problem we have to invert the objective coefficients (change signs)

### Getting current objective coefficients

In [18]:
Example_LP.get_objective(['x','y'])

{'x': 3.0, 'y': 4.0}

### Imposing objective coefficients with changed signs

In [19]:
Example_LP.set_objective({'x':-3,'y':-4})

In [20]:
Example_LP.get_objective(['x','y'])

{'x': -3.0, 'y': -4.0}

### Solve LP again

In [21]:
Example_LP.solve_lp()

### Get new objective value

In [22]:
Example_LP.return_objective_value()

-34.00000000000001

Objective value is now non-zero.
Please note that it is negative, since we minimized the objective function. 
#### Actual objective value is positive and not negative.

### Corresponding optimal solution:

In [23]:
Example_LP.return_primal_values()

{'x': 6.0, 'y': 4.000000000000002}

## What happens if we change the constraint-type of c2 from a >= inequality to an equality? 

In [24]:
Example_LP.get_constraint_types(['c2'])

{'c2': 'G'}

In [25]:
Example_LP.set_constraint_types({'c2': 'E'})

In [26]:
Example_LP.get_constraint_types(['c2'])

{'c2': 'E'}

In [27]:
Example_LP.solve_lp()

In [28]:
Example_LP.return_solution_status()

'optimal'

In [29]:
Example_LP.return_objective_value()

-30.0

In [30]:
Example_LP.return_primal_values()

{'x': 2.0, 'y': 6.0}

## If we also change the constraint-type of c3 from a <= inequality to an equality, the problem becomes infeasible.

In [31]:
Example_LP.get_constraint_types(['c3'])

{'c3': 'L'}

In [32]:
Example_LP.set_constraint_types({'c3': 'E'})

In [33]:
Example_LP.get_constraint_types(['c3'])

{'c3': 'E'}

In [34]:
Example_LP.solve_lp()

In [35]:
Example_LP.return_solution_status()

'infeasible'

### For further operations on the LP (getting/setting variable-bounds, LHS-coefficients, RHS-coefficients ...) please consult the class documentation for:
## rbatools.rba_lp.LinearProblem 